# Geospatial Analysis & Folium Demo

This demo imports geospatial data, transforms it and plots it on a map.

## Useful Links
Pandas documentation: https://pandas.pydata.org/docs/ \
Geopandas documentation: https://geopandas.org/en/stable/getting_started/introduction.html \
Folium documentation: http://python-visualization.github.io/folium/ \
Folium icons: https://fontawesome.com/v4/icons/ \
Folium Choropleth colors: https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3

In [ ]:
# This section imports the core packages we want to use in our analysis

import pandas as pd  # Pandas is a powerful data manipulation and analysis library for Python. It provides data structures like DataFrame to efficiently handle structured data.
import geopandas as gpd  # Geopandas is an extension of pandas that supports geospatial data. It allows for easy manipulation of geometric data types and operations.
import folium  # Folium is a library for creating interactive maps. It is built on the leaflet.js library and allows for easy visualization of geospatial data.

# This section imports custom functions. Importing custom functions allows you to write generic functions and use them as tools in your scripts and analysis.
# This makes the code modular and reusable. Note how the import statements reflect the file path where these functions are stored.
from utils.gis.spatial_joins import join_points_to_polygons, join_polygons, find_nearest_and_distance, filter_intersecting_polygons
from utils.gis.crs_conversion import ensure_crs

Import Data

In [ ]:
# This section creates geospatial dataframes by reading in data from GeoJSON files.
# GeoJSON is a format for encoding a variety of geographic data structures using JavaScript Object Notation (JSON).
# You can load files from all sorts of datasources in both pandas and geopandas.

# Pandas
example_csv_df = pd.read_csv(f"data/example.csv") # Uses the pandas read_csv function to load in data from a csv file

# Geopandas
index_of_multiple_deprivation_df = gpd.read_file(r"data/demo_IMD.geojson") # Uses geopandas read_file function to load data from a geojson
metorlink_lines_df = gpd.read_file(r"data/demo_Metrolink_Lines_Functional.json")
metrolink_stops_df = gpd.read_file(r"data/demo_Metrolink_Stops_Functional.json")

The below sections use Pandas functionality to view and understand your dataframes.

In [ ]:
# Example of viewing the top 10 rows of a data frame. This is useful for debuging and...
index_of_multiple_deprivation_df.head(10)

In [ ]:
# Example that prints the column names of your data frame
index_of_multiple_deprivation_df.columns

In [ ]:
# Example that describes the dataframe and data types
index_of_multiple_deprivation_df.info()

In [ ]:
index_of_multiple_deprivation_df.describe()

In [ ]:
index_of_multiple_deprivation_df

Transform Data

In [ ]:
# Example for creating a copy of a dataframe, you may want to do this when transforming data because...
index_of_multiple_deprivation_df_copy = index_of_multiple_deprivation_df.copy()

# Example for adding a new column to the copied dataframe
index_of_multiple_deprivation_df_copy['new_example_column'] = (index_of_multiple_deprivation_df_copy['Join_Count'] * 5) # This is purly and example to show how you can build new columns using caluclations.

In [ ]:
# Print the new dataframe to see your changes
index_of_multiple_deprivation_df_copy.head(5)

Geospatial Joining using custom function

Plot Data using Folium

In [ ]:
from style_scripts.folium_styles import style_black_bound, style_zone_outline, style_metro_line

In [ ]:
IMD_df = pd.DataFrame(index_of_multiple_deprivation_df) #convert to df
IMD_ranks = (IMD_df['IMD_Rank'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist() #create bins for folium legend 

In [ ]:
m = folium.Map(location=[53.5735, -2.2170], zoom_start=11.5) #base map coordinates

folium.Map(location=[53.5735, -2.2170], zoom_start=12) #base map coordinates

#NMGH Marker - adds marker to the map
NMGH_marker = folium.Marker(
    [53.5174224217791, -2.22877374948477], 
    tooltip='North Manchester Hospital',
    icon=folium.Icon(color='red', icon='hospital-o', prefix='fa')
).add_to(m)

#geojson layers
IMD = f'data/demo_IMD.geojson'
metro_link = f'data/demo_Metrolink_Lines_Functional.json'
metro_stops = f'data/demo_Metrolink_Stops_Functional.json'

cp1 = folium.Choropleth(
    geo_data=IMD,
    name='Index of Deprivation England by LSOA',
    data= IMD_df,
    columns=['OBJECTID_1', 'IMD_Rank'],
    key_on='feature.properties.OBJECTID_1',
    threshold_scale= IMD_ranks,
    fill_color='PuBu',   
    fill_opacity=0.6,
    line_opacity=0.5,
    legend_name='< Less Deprived - More Deprived >',
    highlight=True,
    show=True
).add_to(m)

#code to remove legend
# for key in cp1._children:
#     if key.startswith('color_map'):
#         del(cp1._children[key])
# cp1.add_to(m)

#add tool tip to geojson layer
folium.GeoJsonTooltip(fields=['OBJECTID_1', 'IMD_Rank'], 
aliases=['Object_ID:', 'IMD_Rank:']).add_to(cp1.geojson)


#create feature group for metro geojson data sets
metro_feature_group = folium.FeatureGroup(show=False, name='Metro Link Routes')

#import metro lines and add to feature group
metro_link_lines = (folium.GeoJson(
    metro_link, 
    name='metro link',
    style_function=lambda x:style_metro_line,
)).add_to(metro_feature_group)

#import metro stops and add to feature group
metro_stop_points = (folium.GeoJson(
    metro_stops, 
    name='metro stops',
))

#for each poit in metro_stop_points adjust color and icon
for feature in metro_stop_points.data['features']:
    if feature['geometry']['type'] == 'Point':
        folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
            icon=folium.Icon(icon='train', prefix='fa', icon_size=1, icon_color='#F34200', shadow_size=0, shadow_anchor=0),
            tooltip=feature['properties']['name']
        ).add_to(metro_feature_group)

metro_feature_group.add_to(m)

#adds layer control functionality
folium.LayerControl(autoZIndex=True).add_to(m)

#modify layer order, example: (layer_1, layer_2) - last value goes on top
m.keep_in_front(metro_link_lines)

#adds functionality that lets you click the map and see lat long
m.add_child(folium.LatLngPopup()) 